# Fragmentation for train data
Aquesta part del codi la fem perquè hem trobat problemes per pujar les dades al repositori  

In [1]:
num_art = 30
# Numero d'artistes a agafar

In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
from random import seed # for setting seed
import tensorflow
from IPython import sys_info
import os
from collections import defaultdict
import gc # garbage collection
import shutil

In [3]:
directori_info = '../../all_data_info.csv'
df = pd.read_csv(directori_info, sep=',')
df

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
103245,Jackson Pollock,1948.0,abstract,682.0,220.0,96405.0,wikiart,Action painting,Number 13A (Arabesque),train_and_test,True,25525.jpg
103246,Bernardo Strozzi,NaN,religious painting,329.0,456.0,127594.0,wikiart,Baroque,St. Francis of Assisi,train_only,True,47038.jpg
103247,Josef Sima,NaN,landscape,293.0,512.0,102519.0,wikiart,Surrealism,Maisons à la campagne II,train_and_test,False,7680.jpg
103248,Brett Whiteley,1982.0,marina,293.0,512.0,167423.0,wikiart,NaN,Thebe's Revenge,train_and_test,True,9021.jpg


In [4]:
artist_dict = df['artist'].value_counts().to_dict()
artist_dict

{'Ivan Aivazovsky': 500,
 'John Singer Sargent': 500,
 'Pierre-Auguste Renoir': 500,
 'Marc Chagall': 500,
 'Pablo Picasso': 500,
 'Zdislav Beksinski': 500,
 'Ivan Shishkin': 500,
 'Gustave Dore': 500,
 'Ilya Repin': 500,
 'Giovanni Battista Piranesi': 500,
 'Albrecht Durer': 500,
 'Rembrandt': 500,
 'Paul Cezanne': 499,
 'Pyotr Konchalovsky': 499,
 'Camille Pissarro': 499,
 'Martiros Saryan': 499,
 'Boris Kustodiev': 498,
 'Paul Gauguin': 498,
 'Claude Monet': 498,
 'Raphael Kirchner': 497,
 'Eugene Boudin': 496,
 'Nicholas Roerich': 496,
 'Henri Matisse': 495,
 'Edgar Degas': 495,
 'Vincent van Gogh': 494,
 'Childe Hassam': 493,
 'Camille Corot': 488,
 'Salvador Dali': 485,
 'Theophile Steinlen': 485,
 'Alfred Sisley': 464,
 'Odilon Redon': 451,
 'Isaac Levitan': 449,
 'Fernand Leger': 437,
 'James Tissot': 427,
 'Eyvind Earle': 422,
 'Zinaida Serebriakova': 412,
 'Henri Martin': 405,
 'Ernst Ludwig Kirchner': 388,
 'Francisco Goya': 386,
 'Maurice Prendergast': 379,
 'Peter Paul Rub

In [5]:
df['artist'].nunique()

2319

Per tant hi ha 2319 artistes diferents, i també sabem el numero de vegades que apareix cada artista

In [6]:
artist_dict = dict(sorted(artist_dict.items(), key=lambda item: item[1], reverse=True))
artist_dict

{'Ivan Aivazovsky': 500,
 'John Singer Sargent': 500,
 'Pierre-Auguste Renoir': 500,
 'Marc Chagall': 500,
 'Pablo Picasso': 500,
 'Zdislav Beksinski': 500,
 'Ivan Shishkin': 500,
 'Gustave Dore': 500,
 'Ilya Repin': 500,
 'Giovanni Battista Piranesi': 500,
 'Albrecht Durer': 500,
 'Rembrandt': 500,
 'Paul Cezanne': 499,
 'Pyotr Konchalovsky': 499,
 'Camille Pissarro': 499,
 'Martiros Saryan': 499,
 'Boris Kustodiev': 498,
 'Paul Gauguin': 498,
 'Claude Monet': 498,
 'Raphael Kirchner': 497,
 'Eugene Boudin': 496,
 'Nicholas Roerich': 496,
 'Henri Matisse': 495,
 'Edgar Degas': 495,
 'Vincent van Gogh': 494,
 'Childe Hassam': 493,
 'Camille Corot': 488,
 'Salvador Dali': 485,
 'Theophile Steinlen': 485,
 'Alfred Sisley': 464,
 'Odilon Redon': 451,
 'Isaac Levitan': 449,
 'Fernand Leger': 437,
 'James Tissot': 427,
 'Eyvind Earle': 422,
 'Zinaida Serebriakova': 412,
 'Henri Martin': 405,
 'Ernst Ludwig Kirchner': 388,
 'Francisco Goya': 386,
 'Maurice Prendergast': 379,
 'Peter Paul Rub

In [7]:
artistes = dict(list(artist_dict.items())[:num_art])
print(artistes)
artistes = list(artistes.keys())
print(artistes)

{'Ivan Aivazovsky': 500, 'John Singer Sargent': 500, 'Pierre-Auguste Renoir': 500, 'Marc Chagall': 500, 'Pablo Picasso': 500, 'Zdislav Beksinski': 500, 'Ivan Shishkin': 500, 'Gustave Dore': 500, 'Ilya Repin': 500, 'Giovanni Battista Piranesi': 500, 'Albrecht Durer': 500, 'Rembrandt': 500, 'Paul Cezanne': 499, 'Pyotr Konchalovsky': 499, 'Camille Pissarro': 499, 'Martiros Saryan': 499, 'Boris Kustodiev': 498, 'Paul Gauguin': 498, 'Claude Monet': 498, 'Raphael Kirchner': 497, 'Eugene Boudin': 496, 'Nicholas Roerich': 496, 'Henri Matisse': 495, 'Edgar Degas': 495, 'Vincent van Gogh': 494, 'Childe Hassam': 493, 'Camille Corot': 488, 'Salvador Dali': 485, 'Theophile Steinlen': 485, 'Alfred Sisley': 464}
['Ivan Aivazovsky', 'John Singer Sargent', 'Pierre-Auguste Renoir', 'Marc Chagall', 'Pablo Picasso', 'Zdislav Beksinski', 'Ivan Shishkin', 'Gustave Dore', 'Ilya Repin', 'Giovanni Battista Piranesi', 'Albrecht Durer', 'Rembrandt', 'Paul Cezanne', 'Pyotr Konchalovsky', 'Camille Pissarro', 'Mart

In [8]:
df_filtered = df[df['artist'].isin(artistes)]
df_filtered

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
14,Ivan Aivazovsky,1873,marina,7444.0,5811.0,3189597.0,wikiart,Romanticism,Storm at sea,train_and_test,True,99442.jpg
28,Gustave Dore,1866,religious painting,4915.0,6068.0,29811319.0,wikiart,Romanticism,Death of Eleazer,train_and_test,True,7486.jpg
29,Gustave Dore,NaN,religious painting,4770.0,6219.0,32443139.0,wikiart,Romanticism,The-Deluge,train_and_test,True,35766.jpg
34,Gustave Dore,NaN,illustration,5878.0,4678.0,24158097.0,wikiart,Romanticism,The hoarders and wasters,train_and_test,False,31977.jpg
82,Ivan Aivazovsky,1850,marina,5815.0,3840.0,16019462.0,wikiart,Romanticism,The Ninth Wave,train_and_test,True,81750.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
101978,Raphael Kirchner,1916.0,design,322.0,501.0,26894.0,wikiart,Art Nouveau (Modern),Unknown,train_and_test,True,39044.jpg
102200,Camille Corot,1843,cityscape,496.0,322.0,36142.0,wikiart,Realism,"The Arch of Constantine and the Forum, Rome",train_and_test,False,18645.jpg
102595,Raphael Kirchner,1909.0,illustration,326.0,480.0,35840.0,wikiart,Art Nouveau (Modern),Unknown,train_and_test,False,92800.jpg
102645,Raphael Kirchner,NaN,nude painting (nu),312.0,500.0,27295.0,wikiart,Art Nouveau (Modern),Nude Drawings,train_and_test,True,65191.jpg


In [9]:
artist_dict = {}

for index, row in df_filtered.iterrows():
    artist = row['artist']
    filename = row['new_filename']
    if artist not in artist_dict:
        artist_dict[artist] = []
    artist_dict[artist].append(filename)

print(artist_dict)

artist_lengths = {artist: len(filenames) for artist, filenames in artist_dict.items()}
print(artist_lengths)

{'Ivan Aivazovsky': ['99442.jpg', '81750.jpg', '82140.jpg', '74871.jpg', '51238.jpg', '7390.jpg', '60316.jpg', '67993.jpg', '83695.jpg', '93943.jpg', '4184.jpg', '80901.jpg', '10214.jpg', '91171.jpg', '44523.jpg', '47220.jpg', '25010.jpg', '21538.jpg', '37850.jpg', '75321.jpg', '63271.jpg', '23472.jpg', '10533.jpg', '34320.jpg', '10561.jpg', '27144.jpg', '89386.jpg', '82491.jpg', '97459.jpg', '598.jpg', '17227.jpg', '57702.jpg', '28230.jpg', '85738.jpg', '43628.jpg', '15532.jpg', '25976.jpg', '51694.jpg', '96100.jpg', '61184.jpg', '94542.jpg', '97601.jpg', '23113.jpg', '35443.jpg', '50645.jpg', '53065.jpg', '65446.jpg', '21211.jpg', '59010.jpg', '59147.jpg', '31664.jpg', '89244.jpg', '23402.jpg', '56229.jpg', '18598.jpg', '84934.jpg', '73714.jpg', '92679.jpg', '94324.jpg', '93361.jpg', '7073.jpg', '7714.jpg', '48976.jpg', '82346.jpg', '71438.jpg', '52994.jpg', '11130.jpg', '65715.jpg', '37926.jpg', '90416.jpg', '68117.jpg', '51786.jpg', '50570.jpg', '55311.jpg', '45148.jpg', '55503.jpg

In [10]:
file_list = df_filtered['new_filename'].tolist()
print(len(file_list))
file_list

14878


['99442.jpg',
 '7486.jpg',
 '35766.jpg',
 '31977.jpg',
 '81750.jpg',
 '40672.jpg',
 '95360.jpg',
 '49757.jpg',
 '85018.jpg',
 '5145.jpg',
 '68576.jpg',
 '95021.jpg',
 '9348.jpg',
 '75013.jpg',
 '22289.jpg',
 '67638.jpg',
 '64794.jpg',
 '77220.jpg',
 '71922.jpg',
 '20768.jpg',
 '70088.jpg',
 '35919.jpg',
 '3920.jpg',
 '60634.jpg',
 '96147.jpg',
 '11889.jpg',
 '60161.jpg',
 '856.jpg',
 '44296.jpg',
 '67325.jpg',
 '41682.jpg',
 '37256.jpg',
 '94943.jpg',
 '23452.jpg',
 '42432.jpg',
 '76388.jpg',
 '82140.jpg',
 '83092.jpg',
 '85670.jpg',
 '75187.jpg',
 '19571.jpg',
 '39086.jpg',
 '28179.jpg',
 '76577.jpg',
 '57901.jpg',
 '44603.jpg',
 '97939.jpg',
 '74498.jpg',
 '84023.jpg',
 '7022.jpg',
 '41368.jpg',
 '80565.jpg',
 '14108.jpg',
 '37308.jpg',
 '29842.jpg',
 '55911.jpg',
 '25140.jpg',
 '8328.jpg',
 '3016.jpg',
 '33696.jpg',
 '37447.jpg',
 '72281.jpg',
 '30287.jpg',
 '42728.jpg',
 '28918.jpg',
 '34080.jpg',
 '2186.jpg',
 '59420.jpg',
 '65797.jpg',
 '56641.jpg',
 '4155.jpg',
 '3632.jpg',
 '40

In [11]:
directori = "D:\\"
content = os.listdir(directori)

for c in content:
    if "train_" in c:
        if "train_n_artists" != c:
            directori_folder = os.path.join(directori, c)
            if os.path.isdir(directori_folder):
                print(c)
                files = os.listdir(directori_folder)
                for foto in files:
                    if foto in file_list:
                        rute = os.path.join(directori_folder, foto)
                        rute_final = os.path.join("D:\\train_n_artists", foto)
                        
                        # Verificar si el archivo ya existe en el directorio de destino
                        if not os.path.exists(rute_final):
                            shutil.copy(rute, rute_final)
# Això fa que les imatges passin a la nova carpeta

train_1
train_2
train_3
train_4
train_5
train_6
train_7
train_8
train_9


In [12]:
carpeta_destino = "D:\\train_n_artists"
files_final = os.listdir(carpeta_destino)
list_faltant = []
for f in file_list:
    if f not in files_final:
        list_faltant.append(f)
print(len(files_final))
print(list_faltant)
print(len(list_faltant))

11442
['31977.jpg', '85018.jpg', '68576.jpg', '22289.jpg', '67638.jpg', '35919.jpg', '3920.jpg', '44296.jpg', '85670.jpg', '19571.jpg', '57901.jpg', '97939.jpg', '41368.jpg', '37447.jpg', '30287.jpg', '28918.jpg', '34080.jpg', '2186.jpg', '36944.jpg', '36739.jpg', '8568.jpg', '49442.jpg', '52965.jpg', '112.jpg', '54433.jpg', '30715.jpg', '15109.jpg', '66331.jpg', '52678.jpg', '43301.jpg', '72270.jpg', '82764.jpg', '52003.jpg', '18706.jpg', '46868.jpg', '59086.jpg', '74089.jpg', '94106.jpg', '43269.jpg', '34200.jpg', '7391.jpg', '43440.jpg', '68855.jpg', '79613.jpg', '77437.jpg', '39578.jpg', '19446.jpg', '94593.jpg', '101399.jpg', '64480.jpg', '71902.jpg', '11113.jpg', '7959.jpg', '71756.jpg', '34815.jpg', '96144.jpg', '26488.jpg', '64759.jpg', '2570.jpg', '90093.jpg', '62253.jpg', '44311.jpg', '22331.jpg', '58847.jpg', '20989.jpg', '11288.jpg', '15509.jpg', '36886.jpg', '12944.jpg', '41234.jpg', '32362.jpg', '33351.jpg', '23053.jpg', '85074.jpg', '17412.jpg', '56451.jpg', '57620.jpg',

In [13]:
df_filtered['artist_group'].unique()

array(['train_and_test'], dtype=object)

In [14]:
directori = "D:\\train_n_artists"
content = os.listdir(directori)
content

['100003.jpg',
 '100006.jpg',
 '100031.jpg',
 '100044.jpg',
 '100046.jpg',
 '100051.jpg',
 '100057.jpg',
 '100060.jpg',
 '100082.jpg',
 '100089.jpg',
 '100093.jpg',
 '100106.jpg',
 '100107.jpg',
 '100114.jpg',
 '100116.jpg',
 '100125.jpg',
 '100127.jpg',
 '10013.jpg',
 '100132.jpg',
 '100136.jpg',
 '10015.jpg',
 '100152.jpg',
 '100159.jpg',
 '100189.jpg',
 '100192.jpg',
 '100194.jpg',
 '10021.jpg',
 '100211.jpg',
 '100212.jpg',
 '100222.jpg',
 '100226.jpg',
 '100243.jpg',
 '100281.jpg',
 '100283.jpg',
 '100303.jpg',
 '100318.jpg',
 '100319.jpg',
 '100337.jpg',
 '100376.jpg',
 '100379.jpg',
 '100388.jpg',
 '10039.jpg',
 '100399.jpg',
 '100408.jpg',
 '100423.jpg',
 '100434.jpg',
 '10044.jpg',
 '100447.jpg',
 '100471.jpg',
 '100494.jpg',
 '100496.jpg',
 '100500.jpg',
 '100508.jpg',
 '100510.jpg',
 '100522.jpg',
 '100527.jpg',
 '100542.jpg',
 '100547.jpg',
 '100557.jpg',
 '10056.jpg',
 '100564.jpg',
 '100574.jpg',
 '100584.jpg',
 '10060.jpg',
 '100616.jpg',
 '100621.jpg',
 '100622.jpg',
 '

In [20]:
artist_files = defaultdict(list)
df_filtered = df_filtered[df_filtered['new_filename'].isin(content)]
display(df_filtered)
for index, row in df_filtered.iterrows():
    artist_files[row['artist']].append(row['new_filename'])
artist_files = dict(artist_files)

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
14,Ivan Aivazovsky,1873,marina,7444.0,5811.0,3189597.0,wikiart,Romanticism,Storm at sea,train_and_test,True,99442.jpg
28,Gustave Dore,1866,religious painting,4915.0,6068.0,29811319.0,wikiart,Romanticism,Death of Eleazer,train_and_test,True,7486.jpg
29,Gustave Dore,NaN,religious painting,4770.0,6219.0,32443139.0,wikiart,Romanticism,The-Deluge,train_and_test,True,35766.jpg
82,Ivan Aivazovsky,1850,marina,5815.0,3840.0,16019462.0,wikiart,Romanticism,The Ninth Wave,train_and_test,True,81750.jpg
112,Rembrandt,NaN,portrait,4032.0,5008.0,1384905.0,wikiart,Baroque,Lady with a Lap Dog,train_and_test,True,40672.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
101773,Raphael Kirchner,1917.0,portrait,314.0,520.0,65788.0,wikiart,Art Nouveau (Modern),"Olive Thomas, The Pittsburgh Press",train_and_test,True,95451.jpg
101870,Raphael Kirchner,NaN,portrait,328.0,495.0,55720.0,wikiart,Art Nouveau (Modern),Vivienne Segal,train_and_test,True,38925.jpg
101978,Raphael Kirchner,1916.0,design,322.0,501.0,26894.0,wikiart,Art Nouveau (Modern),Unknown,train_and_test,True,39044.jpg
102645,Raphael Kirchner,NaN,nude painting (nu),312.0,500.0,27295.0,wikiart,Art Nouveau (Modern),Nude Drawings,train_and_test,True,65191.jpg


In [21]:
artist_files


{'Ivan Aivazovsky': ['99442.jpg',
  '81750.jpg',
  '82140.jpg',
  '74871.jpg',
  '51238.jpg',
  '7390.jpg',
  '60316.jpg',
  '83695.jpg',
  '93943.jpg',
  '4184.jpg',
  '80901.jpg',
  '10214.jpg',
  '91171.jpg',
  '44523.jpg',
  '47220.jpg',
  '25010.jpg',
  '21538.jpg',
  '37850.jpg',
  '75321.jpg',
  '63271.jpg',
  '10533.jpg',
  '34320.jpg',
  '10561.jpg',
  '27144.jpg',
  '89386.jpg',
  '82491.jpg',
  '97459.jpg',
  '598.jpg',
  '17227.jpg',
  '28230.jpg',
  '85738.jpg',
  '15532.jpg',
  '25976.jpg',
  '51694.jpg',
  '96100.jpg',
  '61184.jpg',
  '94542.jpg',
  '97601.jpg',
  '23113.jpg',
  '53065.jpg',
  '21211.jpg',
  '59010.jpg',
  '59147.jpg',
  '31664.jpg',
  '89244.jpg',
  '23402.jpg',
  '56229.jpg',
  '18598.jpg',
  '84934.jpg',
  '73714.jpg',
  '92679.jpg',
  '93361.jpg',
  '7073.jpg',
  '7714.jpg',
  '71438.jpg',
  '52994.jpg',
  '11130.jpg',
  '65715.jpg',
  '37926.jpg',
  '90416.jpg',
  '51786.jpg',
  '50570.jpg',
  '55311.jpg',
  '45148.jpg',
  '47512.jpg',
  '15907.jpg

In [22]:
{artist: len(filenames) for artist, filenames in artist_files.items()}

{'Ivan Aivazovsky': 378,
 'Gustave Dore': 387,
 'Rembrandt': 376,
 'Edgar Degas': 387,
 'Claude Monet': 383,
 'Albrecht Durer': 383,
 'Theophile Steinlen': 389,
 'Ivan Shishkin': 367,
 'Giovanni Battista Piranesi': 402,
 'Camille Corot': 387,
 'Pierre-Auguste Renoir': 388,
 'Childe Hassam': 375,
 'Raphael Kirchner': 357,
 'Alfred Sisley': 356,
 'Paul Cezanne': 412,
 'John Singer Sargent': 377,
 'Vincent van Gogh': 392,
 'Zdislav Beksinski': 376,
 'Camille Pissarro': 393,
 'Eugene Boudin': 387,
 'Boris Kustodiev': 388,
 'Nicholas Roerich': 369,
 'Ilya Repin': 397,
 'Martiros Saryan': 399,
 'Pyotr Konchalovsky': 388,
 'Salvador Dali': 348,
 'Pablo Picasso': 377,
 'Henri Matisse': 384,
 'Marc Chagall': 367,
 'Paul Gauguin': 373}